In [1]:
import os

import pandas as pd
import geopandas as gpd

from shapely.geometry import box

import planetary_computer as pc

import find_scenes as fs

In [2]:
years = [2012, 2014, 2016, 2018, 2020]
save = True
verbose = True

### Find NAIP scenes that intersect coastal buffer

In [5]:
# ------------------------------------------------------------------------
# Open SB county coastal zone shapefiles (coarse and actual polygons)

fp = os.path.join(os.getcwd(),'SB_coarse_coastal_buffer','SB_coarse_coastal_buffer.shp')
geom = gpd.read_file(fp)

fp = os.path.join(os.getcwd(),'SB_coastal_buffer','SB_coastal_buffer.shp')
coast = gpd.read_file(fp)

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------

if save:
    # if there is no temp folder, make one
    temp_fp = os.path.join(os.getcwd(), 'temp')
    if os.path.exists(temp_fp) == False:
        os.mkdir(temp_fp)

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------

all_itemids = []

for year in years:
    # ------------------------------------------------------------------------
    # find NAIP scenes intersecting coarse polygon
    coast_scenes = fs.query_geom(geom.geometry[0],year)

    # ------------------------------------------------------------------------
    # find NAIP scenes intersecting actual polygon
    coast_scenes_intersect = []
    for scene in coast_scenes:
        # make bounding box for scene
        s_corners = scene.properties['proj:bbox']
        s_box = box(s_corners[0], s_corners[1], s_corners[2], s_corners[3])

        # match bbox and coast polygon crs
        coast_match = coast.to_crs(scene.properties['proj:epsg']).geometry[0]

        #check intersection and add if True
        if s_box.intersects(coast_match) == True:
            coast_scenes_intersect.append(scene)

    # ------------------------------------------------------------------------
    # extract itemids and create dataframe
    itemids = []
    for item in coast_scenes_intersect:
        itemids.append(item.id)
    itemids_df = pd.DataFrame(itemids, columns = ['itemid'])
    itemids_df['year'] = year
    
    # ------------------------------------------------------------------------
    all_itemids.append(itemids_df)
    
    # ------------------------------------------------------------------------
    if verbose:
        print('************************************************')
        print('year: ', year)
        print('# scenes: ', len(coast_scenes_intersect))
    
    # ------------------------------------------------------------------------
    #  AGGREGATE SCENES BY DATE
    # make dataframe with collected scenes and relevant info to pass to flights_path
    df = pd.DataFrame(coast_scenes_intersect, columns = ['item'])
    df['date'] = df.item.apply(lambda item : item.properties['datetime'])
    df['crs'] = df.item.apply(lambda item : item.properties['proj:epsg'])
    df['bbox'] = df.item.apply(fs.make_bbox)

    # group scenes by date and create shapefile of the scenes bounding boxes merged by date
    flights = fs.flight_paths(df)

    # ------------------------------------------------------------------------
    # save if needed
    if save:
        
        # create folder for flight paths in temp folder
        fp = os.path.join(temp_fp, 'coastal_flights_'+str(year))
        if os.path.exists(fp) == False:
            os.mkdir(fp)
        
        # save grouped scenes as shapefile
        flights.to_file(os.path.join(fp,'coastal_flights_'+str(year)+'.shp'))        

    # ------------------------------------------------------------------------
    if verbose:
        print('************************************************')
        print('year: ', year)
        print('# groups (scenes grouped by date): ', flights.shape[0])
        


        
# ------------------------------------------------------------------------
# save item ids
if save:
    all_years_df = pd.concat(all_itemids, axis=0)
    # save dataframe as csv
    all_years_df.to_csv(os.path.join(os.getcwd(),'coastal_scenes_ids.csv'), index=False)
        


************************************************
year:  2012
# scenes:  38
************************************************
year:  2012
# groups (scenes grouped by date):  3
************************************************
year:  2014
# scenes:  38
************************************************
year:  2014
# groups (scenes grouped by date):  7
************************************************
year:  2016
# scenes:  19
************************************************
year:  2016
# groups (scenes grouped by date):  4
************************************************
year:  2018
# scenes:  25
************************************************
year:  2018
# groups (scenes grouped by date):  4
************************************************
year:  2020
# scenes:  24
************************************************
year:  2020
# groups (scenes grouped by date):  4
